In [63]:
import pandas as pd
import numpy as np
import yfinance as yf

In [54]:
tickerSymbol = 'WEGE3.SA'
tickerData = yf.Ticker(tickerSymbol)

In [56]:
tickerQuarterlyFinancials = tickerData.quarterly_financials
tickerQuarterlyFinancials.T.head()

,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest,Net Income,Selling General Administrative,Gross Profit,Ebit,Operating Income,Other Operating Expenses,...,Non Recurring,Other Items,Income Tax Expense,Total Revenue,Total Operating Expenses,Cost Of Revenue,Total Other Income Expense Net,Discontinued Operations,Net Income From Continuing Ops,Net Income Applicable To Common Shares
,,,,,,,,,,,,,,,,,,,,,
2021-03-31,None,None,9.03717e+08,4.00951e+08,7.64258e+08,6.09209e+08,1.61974e+09,8.93182e+08,8.93182e+08,1.12924e+08,...,None,None,1.24796e+08,5.07688e+09,4.1837e+09,3.45714e+09,1.0535e+07,None,7.78921e+08,7.64258e+08
2020-12-31,None,None,8.69659e+08,3.67133e+08,7.42229e+08,5.89413e+08,1.5963e+09,8.59185e+08,8.59185e+08,1.43456e+08,...,None,None,1.07806e+08,4.88992e+09,4.03073e+09,3.29362e+09,1.0474e+07,None,7.61853e+08,7.42229e+08
2020-09-30,None,None,7.85504e+08,2.77565e+08,6.44246e+08,5.64902e+08,1.51157e+09,8.10636e+08,8.10636e+08,1.36034e+08,...,None,None,1.31892e+08,4.80126e+09,3.99062e+09,3.28969e+09,-2.5132e+07,None,6.53612e+08,6.44246e+08
2020-06-30,None,None,5.73301e+08,2.66371e+08,5.14375e+08,5.02193e+08,1.2321e+09,6.23696e+08,6.23696e+08,1.06214e+08,...,None,None,4.6784e+07,4.06394e+09,3.44025e+09,2.83184e+09,-5.0395e+07,None,5.26517e+08,5.14375e+08


In [57]:
tickerFinancials = tickerData.financials
tickerFinancials.T.head()

,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest,Net Income,Selling General Administrative,Gross Profit,Ebit,Operating Income,Other Operating Expenses,...,Non Recurring,Other Items,Income Tax Expense,Total Revenue,Total Operating Expenses,Cost Of Revenue,Total Other Income Expense Net,Discontinued Operations,Net Income From Continuing Ops,Net Income Applicable To Common Shares
,,,,,,,,,,,,,,,,,,,,,
2020-12-31,None,None,2.74665e+09,3.67133e+08,2.34087e+09,2.16129e+09,5.43751e+09,2.81245e+09,2.81245e+09,4.51649e+08,...,None,None,3.50692e+08,1.74696e+10,1.46571e+10,1.20320e+10,-6.5805e+07,None,2.39596e+09,2.34087e+09
2019-12-31,None,None,1.80445e+09,2.12743e+08,1.61458e+09,1.80157e+09,3.95327e+09,1.8373e+09,1.8373e+09,3.07457e+08,...,None,None,1.71996e+08,1.33474e+10,1.15101e+10,9.39417e+09,-3.2848e+07,None,1.63246e+09,1.61458e+09
2018-12-31,None,None,1.49754e+09,1.38983e+08,1.33832e+09,1.70604e+09,3.46927e+09,1.5036e+09,1.5036e+09,2.53561e+08,...,None,None,1.53394e+08,1.19701e+10,1.04665e+10,8.50082e+09,-6.06e+06,None,1.34415e+09,1.33832e+09
2017-12-31,None,None,1.24045e+09,1.22381e+08,1.14215e+09,1.38303e+09,2.75845e+09,1.18241e+09,1.18241e+09,1.93001e+08,...,None,None,9.9506e+07,9.52383e+09,8.34142e+09,6.76538e+09,5.8036e+07,None,1.14094e+09,1.14215e+09
